<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/TensorFlow_Blocks_and_Layesr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import tensorflow as tf

The code is based on the d2l.ai book http://d2l.ai/

In [61]:
### the code is based on d2l.ai book 

In [62]:
### sequence of models , Dense with relu is the first in the chain
net = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256,activation=tf.nn.relu),
  tf.keras.layers.Dense(10)
])

X=tf.random.uniform((2,20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.21564007,  0.05415054, -0.278714  , -0.37844148,  0.02863857,
         0.4003118 , -0.25837454,  0.1031395 ,  0.39180988,  0.04940575],
       [ 0.13303626, -0.02318302, -0.3375541 , -0.31313676,  0.25510907,
         0.33553824, -0.30773073,  0.20651837,  0.15203333, -0.00993244]],
      dtype=float32)>

In [63]:
net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.21564007,  0.05415054, -0.278714  , -0.37844148,  0.02863857,
         0.4003118 , -0.25837454,  0.1031395 ,  0.39180988,  0.04940575],
       [ 0.13303626, -0.02318302, -0.3375541 , -0.31313676,  0.25510907,
         0.33553824, -0.30773073,  0.20651837,  0.15203333, -0.00993244]],
      dtype=float32)>

In [64]:
### Custom Block

In [65]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.hidden=tf.keras.layers.Dense(units=256,activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(units=10)

  def call(self, X):
    return self.out(self.hidden((X)))

In [66]:
net=MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.15297523, -0.20632286, -0.01411107,  0.12822513, -0.00420676,
         0.24261248, -0.02192844, -0.08237635, -0.1282405 ,  0.08238801],
       [-0.1820739 , -0.26961586, -0.26356012,  0.06556813, -0.03396446,
         0.13208559,  0.09168363,  0.01189797, -0.22588858,  0.02007841]],
      dtype=float32)>

In [67]:
class MySequential(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    for module in self.modules:
      X=module(X)
    return(X)


In [68]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.18045768, -0.24001627, -0.02914331, -0.12118715,  0.43571126,
        -0.22025573, -0.2112957 ,  0.08513686, -0.21573651, -0.1174707 ],
       [-0.17298569,  0.02587445, -0.0734382 , -0.11674695,  0.23802555,
        -0.08251433, -0.14702988,  0.00745137, -0.19311321, -0.32896122]],
      dtype=float32)>

Below the Module is constructed where several models are applied in parallel on the same input data and the results are concatenated. It is intended to use the code in multihead attention in Transformers.

In [69]:
#initialize the list of n models which is constructed from the same model
#
def list_of_models(model, n):
  return [model]*n


In [70]:
list_models = [tf.keras.layers.Dense(10)]*3
arg_list = list_of_models(tf.keras.layers.Dense(10),3)

In [71]:
### use *args to supply a list of models of variable length 
class MyParallel(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    list_res= []
    for module in self.modules:
      list_res.append(module(X))
#      print(len(list_res))
#    print(list_res)
    concat_final= tf.concat(list_res, -1)
    return(concat_final)

In [72]:
net = MyParallel(tf.keras.layers.Dense(10), tf.keras.layers.Dense(10), tf.keras.layers.Dense(10))
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[ 4.7484942e-02,  4.0127447e-01, -2.2112520e-01,  1.1814804e+00,
         2.6646987e-01,  3.0479261e-01,  9.9809527e-02,  5.0731495e-02,
         9.9835509e-01, -8.2024097e-01, -1.1623694e+00, -2.7716348e-01,
        -5.6563544e-01, -6.0920459e-01, -1.0762718e-01, -4.5033857e-02,
         9.0275824e-02,  3.3290938e-02,  1.2171481e+00, -2.1732390e-01,
        -4.6934563e-01,  4.6418119e-02,  8.5590407e-02, -2.1490039e-01,
         6.4405555e-01,  5.1473141e-01, -2.9251280e-01,  1.6549149e-01,
         3.0643564e-01, -4.6193600e-04],
       [ 1.6319886e-01,  2.6040310e-01, -1.4335337e-01,  1.0089786e+00,
        -1.2905097e-01, -7.3949531e-02, -1.4967038e-01, -3.7156228e-02,
         9.2384446e-01, -6.4996088e-01, -8.5700262e-01, -2.6844144e-02,
        -2.1538615e-01, -2.7402422e-01,  4.2066836e-01,  2.8480405e-02,
         1.4840490e-01,  3.8418028e-01,  1.1269141e+00,  2.9616454e-01,
        -3.6861593e-01,  1.7643309e-01, -1.895

In [73]:
### to use the list_models (which is a list) we have to unpack it
net = MyParallel(*list_models)
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[ 0.3830338 , -0.05342612, -1.1095171 , -0.06789342, -0.03647626,
        -0.67386496, -0.07015014,  0.94926023,  0.14623871,  0.2723172 ,
         0.3830338 , -0.05342612, -1.1095171 , -0.06789342, -0.03647626,
        -0.67386496, -0.07015014,  0.94926023,  0.14623871,  0.2723172 ,
         0.3830338 , -0.05342612, -1.1095171 , -0.06789342, -0.03647626,
        -0.67386496, -0.07015014,  0.94926023,  0.14623871,  0.2723172 ],
       [ 0.13784315, -0.18813266, -1.0821737 , -0.6241869 , -0.16280086,
        -0.9862208 , -0.09638233,  0.8489892 ,  0.82189274,  0.8500967 ,
         0.13784315, -0.18813266, -1.0821737 , -0.6241869 , -0.16280086,
        -0.9862208 , -0.09638233,  0.8489892 ,  0.82189274,  0.8500967 ,
         0.13784315, -0.18813266, -1.0821737 , -0.6241869 , -0.16280086,
        -0.9862208 , -0.09638233,  0.8489892 ,  0.82189274,  0.8500967 ]],
      dtype=float32)>

In [74]:
net = MyParallel(*arg_list)
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-0.4385905 , -0.04031485, -0.32234856, -0.02818382, -0.42702287,
         1.5153399 , -0.5282389 , -0.0982194 ,  0.1974664 , -0.9258027 ,
        -0.4385905 , -0.04031485, -0.32234856, -0.02818382, -0.42702287,
         1.5153399 , -0.5282389 , -0.0982194 ,  0.1974664 , -0.9258027 ,
        -0.4385905 , -0.04031485, -0.32234856, -0.02818382, -0.42702287,
         1.5153399 , -0.5282389 , -0.0982194 ,  0.1974664 , -0.9258027 ],
       [-0.44829634,  0.11966628, -0.4454378 ,  0.03178519, -0.05969727,
         1.3988982 , -0.3383227 , -0.18196534,  0.38020462, -0.02946788,
        -0.44829634,  0.11966628, -0.4454378 ,  0.03178519, -0.05969727,
         1.3988982 , -0.3383227 , -0.18196534,  0.38020462, -0.02946788,
        -0.44829634,  0.11966628, -0.4454378 ,  0.03178519, -0.05969727,
         1.3988982 , -0.3383227 , -0.18196534,  0.38020462, -0.02946788]],
      dtype=float32)>

In [75]:
### Nested Models 
class NestedBlocks(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.net=tf.keras.Sequential()
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.dense = tf.keras.layers.Dense(20,activation = tf.nn.relu)

  def call(self, inputs):
    return self.dense(self.net(inputs))



In [76]:
net = NestedBlocks()
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.08824383, 0.        , 0.251317  , 0.10667011, 0.13481522,
        0.        , 0.        , 0.        , 0.        , 0.24866459,
        0.        , 0.01884621, 0.12187549, 0.0675218 , 0.22904147,
        0.        , 0.        , 0.17180899, 0.08450582, 0.        ],
       [0.02093283, 0.        , 0.20381552, 0.        , 0.20379037,
        0.        , 0.20165554, 0.        , 0.11114582, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.49453607, 0.12784421]],
      dtype=float32)>

Parameter access

In [77]:
import numpy as np

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.6715543],
       [-0.804057 ]], dtype=float32)>

In [78]:
print(net.layers[2].weights)

[<tf.Variable 'dense_66/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.90698975],
       [ 1.0091729 ],
       [-0.8561751 ],
       [-0.934435  ]], dtype=float32)>, <tf.Variable 'dense_66/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [79]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_66/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [80]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_65/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.6931675 , -0.3968479 , -0.7143742 ,  0.74970204],
       [-0.83552754, -0.31706995,  0.00711459,  0.8453565 ],
       [-0.57526433, -0.75424415,  0.33068687,  0.0733915 ],
       [-0.21450806, -0.5957988 , -0.0084393 , -0.14078873]],
      dtype=float32)>, <tf.Variable 'dense_65/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.6931675 , -0.3968479 , -0.7143742 ,  0.74970204],
       [-0.83552754, -0.31706995,  0.00711459,  0.8453565 ],
       [-0.57526433, -0.75424415,  0.33068687,  0.0733915 ],
       [-0.21450806, -0.5957988 , -0.0084393 , -0.14078873]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-0.90698975],
       [ 1.0091729 ],
       [-0.8561751 ],
       [-0.934435  ]], dtype=float32), array([0.], dtype=float32)]


In [81]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

In [82]:
### collecting parameters from nested blocks
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00599643],
       [0.00359134]], dtype=float32)>

In [83]:
print(rgnet.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_14 (Sequential)   (2, 4)                    80        
_________________________________________________________________
dense_71 (Dense)             (2, 1)                    5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [84]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'sequential_14/block-1/dense_68/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [85]:
net_prl = MyParallel(tf.keras.layers.Dense(4), tf.keras.layers.Dense(4), tf.keras.layers.Dense(4))
X = tf.random.uniform((1,4))
net_prl(X)

<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[ 0.6436592 , -0.06759639, -0.11064615,  0.15265562, -0.07731043,
         0.6013583 , -0.03865418,  0.01184419,  0.43359563,  0.08102705,
        -0.12206343, -0.4855148 ]], dtype=float32)>

In [86]:
print(net_prl.summary())

Model: "my_parallel_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             multiple                  20        
_________________________________________________________________
dense_73 (Dense)             multiple                  20        
_________________________________________________________________
dense_74 (Dense)             multiple                  20        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________
None
